# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins":/plugins -v "C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* neo4j`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
# We assigned a different port in order to be able to run two Neo4J dockers at the same time
NEO4J_URI="neo4j://localhost:8687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>,
 <Record label='bdmmREVIEWS'>,
 <Record label='bdmmUSER'>,
 <Record label='bdmmBEERS'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>,
 <Record relationshipType='bdmmREVIEWED'>,
 <Record relationshipType='bdmmPOSTED'>]


# Submission

GROUP NUMBER:

**17**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20220609|Samuel Santos|
|20221385|Felix Gaber|
|XXXXXX|XXXXXX|
|XXXXXX|XXXXXX|

In [7]:
# In several questions we want to filter for beers that are still in production.
# In all cases, we what to check his variable will be used in those cases.

retired = 'f'


#### 1. How many different countries exist in the database?

In [8]:
# Count nodes of type COUNTRIES

query = """
        MATCH (c:COUNTRIES)
        RETURN count(c) AS number_countries
    """

result = execute_read(driver, query)

pprint(result)

[<Record number_countries=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [9]:
# Count nodes of type REVIEWS that are connected to nodes of type BEER.
# Order by the count, descending, and return just the first one

query = """
MATCH (b:BEERS)-[:REVIEWED]->(r:REVIEWS)
RETURN b.name, b.id, COUNT(r) AS number_reviews
ORDER BY number_reviews DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record b.name='Breakfast Stout' b.id='11757' number_reviews=17160>]


In [10]:
# Same logic, but for breweries

query = """
MATCH (br:BREWERIES)-[]->(b:BEERS)-[]->(r:REVIEWS)
RETURN br.name, br.id, COUNT(r) AS number_reviews
ORDER BY number_reviews DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record br.name='Sierra Nevada Brewing Co.' br.id='140' number_reviews=175161>]


In [11]:
# Same logic, but for countries, accounting for the extra relations involved

query = """
MATCH (c:COUNTRIES)<-[]-(:CITIES)<-[]-(br:BREWERIES)-[]->(b:BEERS)-[]->(r:REVIEWS)
RETURN c.name, COUNT(r) AS number_reviews
ORDER BY number_reviews DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record c.name='US' number_reviews=7675804>]


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [12]:
# Connect user CTJman to other users, through the BEERs that each of them reviewed.
# Return the top count

username = 'CTJman'

query = f"""
MATCH (u1:USER {{name: '{username}' }})<-[:POSTED]-(r1:REVIEWS)<-[:REVIEWED]-(b:BEERS)-[:REVIEWED]->(r2:REVIEWS)-[:POSTED]->(u2:USER)
WITH u2, count(r2) as num_shared_reviews
ORDER BY num_shared_reviews DESC
RETURN u2.name AS user, num_shared_reviews
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record user='acurtis' num_shared_reviews=1428>]


#### 4. Which Portuguese brewery has the most beers?


In [13]:
# Connect countries to beers, filter by country = PT, then
# count beers by brewery and return the one with the highest count.

country = 'PT'

query = f"""
MATCH (c:COUNTRIES {{name: '{country}'}})<-[]-(:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(beer:BEERS) 
RETURN b.name AS brewery, count(beer) as num_beers
ORDER BY num_beers DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record brewery='Dois Corvos Cervejeira' num_beers=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [14]:
# Start with the previous query to get the top brewery
# Connect the beers from this brewery to the reviews, and
# return the beer that has the most reviews

query = f"""
MATCH (c:COUNTRIES {{name:'{country}'}})<-[:IN]-(:CITIES)<-[:IN]-(b:BREWERIES)-[:BREWED]->(beer:BEERS)
WITH b, count(beer) as num_beers
ORDER BY num_beers DESC
LIMIT 1
MATCH (b)-[:BREWED]->(beer:BEERS)-[:REVIEWED]->(r:REVIEWS)
RETURN beer.name AS beer, count(r) as num_reviews
ORDER BY num_reviews DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record beer='Finisterra' num_reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?

In [15]:
# We included only non-retired beers, because the question is asked in present tense (not past tense)
# Connect breweries to styles, through the relation with beer.
# group by brewery, count the number of beer styles, then average that count

query = f"""
MATCH (b:BREWERIES)-[:BREWED]->(beer:BEERS {{retired:'{retired}'}})-[:HAS_STYLE]->(s:STYLE)
WITH b, count(DISTINCT s.name) AS num_styles
RETURN avg(num_styles) AS avg_num_styles
    """

result = execute_read(driver, query)

pprint(result)

[<Record avg_num_styles=9.332362897765716>]


#### 7. Which brewery produces the strongest beers according to ABV?

In [16]:
# we included only non-retired beers, because the question is asked in present tense (not past tense)
# We interpret that the brewery that produces the strongest beers is the one that has
# a higher average rate in ABV (they may not produce the strongest beer, though)

query = f"""
MATCH (b:BREWERIES)-[:BREWED]->(beer:BEERS{{retired:'{retired}'}})
WHERE beer.abv <> 'nan'
RETURN b.name AS brewery, avg(toFloat(beer.abv)) as avg_abv
ORDER BY avg_abv DESC
LIMIT 1
    """

result = execute_read(driver, query)

pprint(result)

[<Record brewery='1648 Brewing Company Ltd' avg_abv=51.949999999999996>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [17]:
# we interpret aroma as "smell" and apperance as "look". So, we want to maximize their sum.
# We checked what's the style whose beers average a higher sum of look and smell is highest;
# we will want to try some beer of this style.

query = """
MATCH (s:STYLE)<-[:HAS_STYLE]-(b:BEERS)-[:REVIEWED]->(r:REVIEWS)
WHERE r.look <> 'nan' AND r.smell <> 'nan'
WITH s.name as beer_style, avg(toFloat(r.look) + toFloat(r.smell)) as aroma_appearence_score
RETURN beer_style, aroma_appearence_score
ORDER BY aroma_appearence_score DESC
LIMIT 1
"""

result = execute_read(driver, query)

pprint(result)



[<Record beer_style='New England IPA' aroma_appearence_score=8.797210377972341>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

##### Question 9.1

In [18]:
# Step 0 - Clear graph

try:
    query = """
            CALL gds.graph.drop('node_similarity_algo') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `node_similarity_algo` does not exist on database `neo4j`. It might exist on another database.')


In [19]:
# Step 1 - Create an appropriate graph
# Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
# We connected countries to styles, through breweries and beers.
# We filtered out countries whose name is 'nan'
# We then grouped the results by country and style, and counted the number of distinct beers
# for that combination of country + style (nbeers_per_style).
# We then created a list with the 10 styles with the highest number of beers
# This result presented a group of up to 10 styles associated with the country.
# We used UNWIND to unfold this records, creating a list with country + style
# To prepare for the algorithm, we then extract the IDs for
# both country as source and style as target


try:
    query = """
        CALL gds.graph.project.cypher(
            'node_similarity_algo',
            "MATCH (n) where head(Labels(n))='COUNTRIES' or head(Labels(n))='STYLE' RETURN id(n) AS id",
            
            "MATCH (c1:COUNTRIES)<-[]-(:CITIES)<-[]-(:BREWERIES)-[]->(b1:BEERS)-[]->(s1:STYLE)
            WHERE c1.name <> 'nan'
            WITH c1, s1, count(distinct  b1.id) as nbeers_per_style
            ORDER BY c1, nbeers_per_style desc
            WITH c1, COLLECT(s1)[0..10] as top10_1

            UNWIND top10_1 as styleb

            return id(c1) as source, id(styleb) as target"
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery="MATCH (n) where head(Labels(n))='COUNTRIES' or head(Labels(n))='STYLE' RETURN id(n) AS id" relationshipQuery="MATCH (c1:COUNTRIES)<-[]-(:CITIES)<-[]-(:BREWERIES)-[]->(b1:BEERS)-[]->(s1:STYLE)\n            WHERE c1.name <> 'nan'\n            WITH c1, s1, count(distinct  b1.id) as nbeers_per_style\n            ORDER BY c1, nbeers_per_style desc\n            WITH c1, COLLECT(s1)[0..10] as top10_1\n\n            UNWIND top10_1 as styleb\n\n            return id(c1) as source, id(styleb) as target" graphName='node_similarity_algo' nodeCount=313 relationshipCount=1408 projectMillis=15302>]


In [20]:
# Step 2 - Run the algorithm
# We identified that there are more than two countries with a top similarity of 1.0.
# We investivated and confirmed that, indeed, these are countries whose top-10 list of styles
# includes the exact same styles (we understood that there was no requirement for the
# list to be in the exact same order, just to include the same styles)


try:
    query = """
        CALL gds.nodeSimilarity.stream('node_similarity_algo')
            YIELD node1, node2, similarity
            with gds.util.asNode(node1).name AS Country1, gds.util.asNode(node2).name AS Country2, similarity
            RETURN Country1, Country2, similarity
            ORDER BY similarity DESCENDING
            limit 1
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record Country1='AU' Country2='NZ' similarity=1.0>]


#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

##### 10.2

In [21]:
# Step 0 - Clear graph

try:
    query = """
            CALL gds.graph.drop('most_infl_user') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `most_infl_user` does not exist on database `neo4j`. It might exist on another database.')


In [22]:
# We connected users to reviews, excluding users with name 'nan'
# We then obtained the list of IDs for reviews as source, and users as target

try:
    query = """
        CALL gds.graph.project.cypher(
        'most_infl_user',
        "MATCH (n) where head(Labels(n))='USER' or head(Labels(n))='REVIEWS' RETURN id(n) AS id",
        
        "MATCH (u1:USER)<-[:POSTED]-(r1:REVIEWS)
        WHERE u1.name <> 'nan'
        RETURN id(r1) AS source, id(u1) AS target"
        );
        """

    result = execute_read(driver, query)

    pprint(result)
    
except Exception as e:
    pprint(e)

[<Record nodeQuery="MATCH (n) where head(Labels(n))='USER' or head(Labels(n))='REVIEWS' RETURN id(n) AS id" relationshipQuery="MATCH (u1:USER)<-[:POSTED]-(r1:REVIEWS)\n        WHERE u1.name <> 'nan'\n        RETURN id(r1) AS source, id(u1) AS target" graphName='most_infl_user' nodeCount=9238044 relationshipCount=9069294 projectMillis=35733>]


In [23]:
# We used the pageRank algorithm to check which user is most influential
# in terms of its reviews.

try:
    query = """
    CALL gds.pageRank.stream('most_infl_user') 
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS user, score
    ORDER BY score DESC
    LIMIT 1;
    """

    result = execute_read(driver, query)

    pprint(result)
    
except Exception as e:
    pprint(e)

[<Record user='Sammy' score=1759.2675000004458>]


#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [24]:
# We can only try beers that are still being produced

# As no specific preference has been indicated, the selection of recommended beers will be
# based solely on the "score" of the reviews.
# To ensure greater accuracy, we have chosen to use the "score" metric rather than 
# the rounded "overall" metric, which is rounded to the next quarter.
# Additionally, we have limited our selection to the top 100 beers by number
# of reviews to avoid any bias from beers with only a few reviews.

query = f"""
    MATCH (b:BEERS {{retired:'{retired}'}})-[rv:REVIEWED]->(r:REVIEWS)
    WITH b as top100, count(r.id) as cnt
    order by cnt Desc
    LIMIT 100

    MATCH (top100)-[rvi:REVIEWED]->(re:REVIEWS)
    where re.score <> 'nan' 
    RETURN top100.name, avg(toFloat(re.score)) as avgscore, count(*)
    order by avgscore Desc
    LIMIT 3
    """

result = execute_read(driver, query)

pprint(result)

[<Record top100.name='Heady Topper' avgscore=4.713771880688968 count(*)=14282>,
 <Record top100.name='Bourbon County Brand Coffee Stout' avgscore=4.674392905866312 count(*)=7330>,
 <Record top100.name='Pliny The Elder' avgscore=4.6447993568701005 count(*)=14927>]
